In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.9 MB/s eta 0:00:00


In [2]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.8 MB/s eta 0:00:00


In [3]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.5 MB/s eta 0:00:00


In [4]:
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Define your model and tokenizer
model_name = "distilbert-base-cased"
model = DistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load dataset
df = pd.read_csv("/content/gdrive/MyDrive/fake_news/FakeNewsNet_DS/gossipcop_cleaned.csv")
df['SECTION_CLEANED'] = df['SECTION_CLEANED'].astype(str)

# Randomly select  % of the data
df_data = df.sample(frac=1.0, random_state=42)


# Split the dataset into training, validation, and test sets
# First, split into training and temp sets (80% training, 20% temp)
train_df, temp_df = train_test_split(df_data, test_size=0.20, random_state=42)

# Then, split the temp set into validation and test sets (50% validation, 50% test)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Optionally,reset the index of the DataFrames
train_df = train_df.reset_index(drop=True)
validation_df = validation_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


In [ ]:
df.groupby("label").describe()

Unnamed: 0                                                            \
           count     mean          std      min      25%      50%      75%   
label                                                                        
0         5323.0  19478.0  1536.762072  16817.0  18147.5  19478.0  20808.5   
1        16817.0   8408.0  4854.794074      0.0   4204.0   8408.0  12612.0   

               word count                                                   
           max      count       mean       std  min  25%   50%   75%   max  
label                                                                       
0      22139.0     5323.0  11.057862  3.917703  1.0  9.0  11.0  14.0  31.0  
1      16816.0    16817.0  11.299875  3.878651  1.0  9.0  11.0  14.0  39.0

In [ ]:
train_df.groupby("label").describe()

Unnamed: 0                                                        \
           count         mean          std      min       25%      50%   
label                                                                    
0         4238.0  19482.26168  1539.210334  16817.0  18143.25  19498.5   
1        13474.0   8420.56212  4847.687893      0.0   4248.25   8413.5   

                         word count                                       \
            75%      max      count       mean       std  min  25%   50%   
label                                                                      
0      20801.75  22139.0     4238.0  11.027843  3.943509  1.0  9.0  11.0   
1      12630.75  16816.0    13474.0  11.290931  3.869135  1.0  9.0  11.0   

                   
        75%   max  
label              
0      14.0  31.0  
1      14.0  39.0

In [ ]:
validation_df.groupby("label").describe()

Unnamed: 0                                                        \
           count          mean          std      min      25%      50%   
label                                                                    
0          544.0  19460.169118  1531.983748  16828.0  18178.0  19395.5   
1         1670.0   8535.662275  4873.111339      1.0   4245.5   8712.0   

                         word count                                       \
            75%      max      count       mean       std  min  25%   50%   
label                                                                      
0      20911.25  22138.0      544.0  11.108456  3.826967  1.0  9.0  11.0   
1      12736.50  16813.0     1670.0  11.241916  3.917228  1.0  9.0  11.0   

                   
        75%   max  
label              
0      13.0  26.0  
1      14.0  30.0

In [ ]:
test_df.groupby("label").describe()

Unnamed: 0                                                        \
           count          mean          std      min      25%      50%   
label                                                                    
0          541.0  19462.545287  1524.906059  16832.0  18149.0  19379.0   
1         1673.0   8179.393903  4889.282976      2.0   3915.0   8019.0   

                        word count                                             \
           75%      max      count       mean       std  min  25%   50%   75%   
label                                                                           
0      20792.0  22131.0      541.0  11.242144  3.804792  2.0  9.0  11.0  14.0   
1      12395.0  16814.0     1673.0  11.429767  3.915984  1.0  9.0  11.0  14.0   

             
        max  
label        
0      28.0  
1      38.0

In [ ]:
df.head()

,Unnamed: 0,id,title,label,word count,SECTION_CLEANED
0,0,gossipcop-882573,Teen Mom Star Jenelle Evans' Wedding Dress Is ...,1,12,Teen Mom Star Jenelle Evans Wedding Dress Is A...
1,1,gossipcop-875924,Kylie Jenner refusing to discuss Tyga on Life ...,1,10,Kylie Jenner refuse discuss Tyga Life Kylie
2,2,gossipcop-894416,Quinn Perkins,1,2,Quinn Perkins
3,3,gossipcop-857248,I Tried Kim Kardashian's Butt Workout & Am For...,1,10,I Tried Kim Kardashians Butt Workout Am Foreve...
4,4,gossipcop-884684,Celine Dion donates concert proceeds to Vegas ...,1,9,Celine Dion donate concert proceed Vegas shoot...


In [ ]:
train_texts = list(train_df['SECTION_CLEANED'])
len(train_texts)

17712

In [ ]:
train_labels = list(train_df['label'])
len(train_labels)

17712

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Tokenize the dataset
train_encodings = tokenizer(train_texts, return_tensors="pt", padding=True, truncation=True,max_length=128)

In [ ]:
train_dataset = CustomDataset(train_encodings, train_labels)

In [ ]:
val_texts = list(validation_df['SECTION_CLEANED'])
len(val_texts)

2214

In [ ]:
val_labels = list(validation_df['label'])
len(val_labels)

2214

In [ ]:
# Tokenize the dataset
val_encodings = tokenizer(val_texts, return_tensors="pt", padding=True, truncation=True,max_length=128)

In [ ]:
# Define a custom dataset for validation
val_dataset = CustomDataset(val_encodings, val_labels)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./distilbert_fake_news_classification_model",
    evaluation_strategy="steps",
    num_train_epochs=7,
    save_steps=500,
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    warmup_steps=399,
    weight_decay=0.009437022954011717,
    logging_dir="./logs",
    logging_steps=500,
)

In [ ]:
# Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.538400,0.627535
1000,0.443400,0.412983
1500,0.446000,0.434202
2000,0.434700,0.415630
2500,0.375100,0.464246
3000,0.380500,0.452808
3500,0.384200,0.400656
4000,0.359600,0.492966
4500,0.363800,0.531595
5000,0.309200,0.558717


TrainOutput(global_step=15498, training_loss=0.2769389687268745, metrics={'train_runtime': 1407.874, 'train_samples_per_second': 88.065, 'train_steps_per_second': 11.008, 'total_flos': 2405835637934400.0, 'train_loss': 0.2769389687268745, 'epoch': 7.0})

In [ ]:
# Save the model if needed
trainer.save_model("/content/gdrive/MyDrive/fake_news/FakeNewsNet_DS/Models/DistilBERT/gossicop_nlr_7a")

In [ ]:
results = trainer.evaluate(val_dataset)

In [ ]:
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 0.7785927653312683, 'eval_runtime': 4.5743, 'eval_samples_per_second': 484.013, 'eval_steps_per_second': 121.113, 'epoch': 7.0}


#Validation

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Load the model for inference
loaded_model = DistilBertForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/fake_news/FakeNewsNet_DS/Models/DistilBERT/gossicop_nlr_7a")


In [ ]:
val_labels = torch.tensor(val_labels)

In [ ]:
# Make predictions on the validation set
with torch.no_grad():
    validation_outputs = loaded_model(**val_encodings)
    val_logits = validation_outputs.logits
    val_predicted_labels = torch.argmax(val_logits, dim=1)

In [ ]:
# Calculate evaluation metrics for the validation set
accuracy_val = accuracy_score(val_labels, val_predicted_labels)
precision_val = precision_score(val_labels, val_predicted_labels)
recall_val = recall_score(val_labels, val_predicted_labels)
f1_val = f1_score(val_labels, val_predicted_labels)

In [ ]:
print(f"Validation Accuracy: {accuracy_val*100:.4f}")
print(f"Validation Precision: {precision_val*100:.4f}")
print(f"Validation Recall: {recall_val*100:.4f}")
print(f"Validation F1 Score: {f1_val*100:.4f}")

Validation Accuracy: 83.9205
Validation Precision: 86.8687
Validation Recall: 92.6946
Validation F1 Score: 89.6871


#Method 2


In [ ]:
#Alternate
# Inference
def get_predictions(model, input_data):
    with torch.no_grad():
        output = model(**input_data)
    logits = output.logits
    probabilities = torch.softmax(logits, dim=1)
    return probabilities


In [ ]:
#Alternate
distilbert_predictions = [get_predictions(loaded_model, data) for data in val_encodings]


TypeError: ignored

In [ ]:
#Alternate
final_predictions_val = [torch.argmax(distil_probs, dim=1).item() for distil_probs in distilbert_predictions]

In [ ]:
# Calculate evaluation metrics for the validation set
accuracy_val = accuracy_score(val_labels, final_predictions_val)
precision_val = precision_score(val_labels, final_predictions_val)
recall_val = recall_score(val_labels, final_predictions_val)
f1_val = f1_score(val_labels, final_predictions_val)

In [ ]:
print(f"Validation Accuracy: {accuracy_val*100:.4f}")
print(f"Validation Precision: {precision_val*100:.4f}")
print(f"Validation Recall: {recall_val*100:.4f}")
print(f"Validation F1 Score: {f1_val*100:.4f}")

#Testing

In [ ]:
test_texts = list(test_df['SECTION_CLEANED'])
len(test_texts)

2214

In [ ]:
test_labels = list(test_df['label'])
len(test_labels)

2214

In [ ]:
test_encodings = tokenizer(test_texts, return_tensors="pt", padding=True, truncation=True,max_length=128)

In [ ]:
test_labels = torch.tensor(test_labels)

In [ ]:
# Make predictions on the test set
with torch.no_grad():
    test_outputs = loaded_model(**test_encodings)
    test_logits = test_outputs.logits
    test_predicted_labels = torch.argmax(test_logits, dim=1)

In [ ]:
# Calculate evaluation metrics for the test set
test_accuracy = accuracy_score(test_labels, test_predicted_labels)
test_precision = precision_score(test_labels, test_predicted_labels)
test_recall = recall_score(test_labels, test_predicted_labels)
test_f1 = f1_score(test_labels, test_predicted_labels)

In [ ]:
print("\nTest Set Metrics:")
print(f"Test Accuracy: {test_accuracy*100:.4f}")
print(f"Test Precision: {test_precision*100:.4f}")
print(f"Test Recall: {test_recall*100:.4f}")
print(f"Test F1 Score: {test_f1*100:.4f}")


Test Set Metrics:
Test Accuracy: 85.8627
Test Precision: 88.2022
Test Recall: 93.8434
Test F1 Score: 90.9354


#Method 2

In [ ]:
#Alternate
# Inference
def get_predictions(model, input_data):
    with torch.no_grad():
        output = model(**input_data)
    logits = output.logits
    probabilities = torch.softmax(logits, dim=1)
    return probabilities


In [ ]:
#Alternate
distilbert_predictions_test = [get_predictions(loaded_model, data) for data in test_encodings]


In [ ]:
#Alternate
final_predictions_test = [torch.argmax(distil_probs, dim=1).item() for distil_probs in distilbert_predictions_test]

In [ ]:
# Calculate evaluation metrics for the test set
test_accuracy = accuracy_score(test_labels, final_predictions_test)
test_precision = precision_score(test_labels, final_predictions_test)
test_recall = recall_score(test_labels, final_predictions_test)
test_f1 = f1_score(test_labels, final_predictions_test)

In [ ]:
print("\nTest Set Metrics:")
print(f"Test Accuracy: {test_accuracy*100:.4f}")
print(f"Test Precision: {test_precision*100:.4f}")
print(f"Test Recall: {test_recall*100:.4f}")
print(f"Test F1 Score: {test_f1*100:.4f}")

In [7]:
# Define your model and tokenizer
model_name = "distilbert-base-cased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)


In [6]:
text_test = "A Cheetah is roaming at Kongu Engineering College in Perundurai"

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [9]:
# Load the model for inference
loaded_model = DistilBertForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/fake_news/FakeNewsNet_DS/Models/DistilBERT/gossicop_nlr_7a")


In [10]:
test_encodings = tokenizer(text_test, return_tensors="pt", padding=True, truncation=True,max_length=128)

In [11]:
# Make predictions on the test set
with torch.no_grad():
    test_outputs = loaded_model(**test_encodings)
    test_logits = test_outputs.logits
    test_predicted_labels = torch.argmax(test_logits, dim=1)

In [12]:
test_predicted_labels

tensor([1])

In [13]:
output = test_predicted_labels

# Check the value and display the appropriate message
if output.item() == 0:
    print("Fake News")
elif output.item() == 1:
    print("Real News")

Real News
